# Устанавливаем java и ChromHmm

In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/fix-colab-gpu.sh && bash fix-colab-gpu.sh

In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/colab_build.sh && bash colab_build.sh

In [ ]:
!java --list-modules | grep "jdk.jshell"


In [ ]:
! wget http://compbio.mit.edu/ChromHMM/ChromHMM.zip

In [ ]:
!unzip /content/ChromHMM.zip

## Скачиваем файлы

Пример

In [ ]:
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H3k04me1Dex100nmAlnRep1.bam -O H3K04me1.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H3k04me2Dex100nmAlnRep1.bam -O H3K04me2.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H3k04me3Dex100nmAlnRep1.bam -O H3K04me3.bam

!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H3k09acEtoh02AlnRep1.bam -O H3K09ac.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H3k09me3Etoh02AlnRep1.bam -O H3K09me3.bam

!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H4k20me1Etoh02AlnRep1.bam -O H4K20me1.bam

!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H3k27acDex100nmAlnRep1.bam -O H3K27ac.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H3k27me3Dex100nmAlnRep1.bam -O H3K27me3.bam

!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H3k36me3Dex100nmAlnRep1.bam -O H3K36me3.bam
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549H3k79me2Dex100nmAlnRep1.bam -O H3K79me2.bam


In [ ]:
!wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeBroadHistone/wgEncodeBroadHistoneA549ControlEtoh02AlnRep1.bam -O Control.bam

In [ ]:
!touch cellmarkfiletable.txt

* Вручную создаем текстовый файл cellmarkfiletable.txt, в котором указываем название типа клеток, разных гистоновых меток, а также соответствующие .bam файлы для эксперимента и контроля. Один и тот же контрольный файл .bam может быть использован для всех экспериментов.

* Более подробно о содержании файла cellmarkfiletable.txt можно посмотреть в руководстве пользователя ChromHMM

* Файл cellmarkfiletable.txt добавляем в github репозиторий для отчета

Запускаем ChromHMM с опцией BinarizeBam, чтобы конвертировать профили из ChIP-seq экспериментов (bam-файлы) в табличку из 0 и 1, т.е. чтобы сделать разбивку генома на условные интервалы (бины) длиной 200 п.о.
Для 11 меток программа работает около 5 мин


In [ ]:
import os

# !> cellmarkfiletable.txt
control = 'Control.bam'

with open(f'cellmarkfiletable.txt', 'a') as the_file:
  for file in os.listdir():
    if file[-3:] == 'bam' and "Control" not in file:
      the_file.write(f'A549\t{file[:-4]}\t{file}\t{control}\n')

In [ ]:
!java -mx5000M -jar /content/ChromHMM/ChromHMM.jar BinarizeBam -b 200  /content/ChromHMM/CHROMSIZES/hg19.txt /content/ cellmarkfiletable.txt   binarizedData


Запускаем ChromHMM с опцией LearnModel (см. руководство пользователя), которая автоматически определит параметры N разных эпигенетических типов с наиболее выраженными наборами гистоновых меток и присвоит каждому геномному интервалу определенный эпигенетический тип. Количество разных эпигенетических типов (или состояний) выбираете сами (например 10 штук).
Не забудьте, что мы работаем с версией референсного генома человека hg19
Для 11 меток программа работает около 10 минут


In [ ]:
!java -mx5000M -jar /content/ChromHMM/ChromHMM.jar LearnModel -b 200 /content/binarizedData/ /content/data 10 hg19

В результате выполнения команды LearnModel будет создан набор файлов (в том числе HTML страничка) в папке выдачи, которая была указана при запуске. Скачиваем эту папку себе на копьютер и открываем HTML файл.

Дальнейшие инстркции -- см. в описании ДЗ

In [ ]:
!touch A549_10_dense_new.bed


In [ ]:
states = [
    'Heterochromatin', 
    'Repressed', 
    'Weak_Txn', 
    'Weak_Txn', 
    'Weak_Txn', 
    'Weak_Enhancer', 
    'Weak_Enhancer', 
    'Strong_Enhancer', 
    'Active_Promoter', 
    'Active_Promoter'
]

with open(f'A549_10_dense_new.bed', 'a') as new_f:
    with open(f'data/A549_10_dense.bed', 'r') as old_f:
        lines = old_f.readlines()
        new_f.write(lines[0])
        for line in lines[1:]:
            l = line.split('\t')
            l[3] += ('_' + states[int(l[3]) - 1])
            new_f.write('\t\t'.join(l))



In [ ]:
!head A549_10_dense_new.bed